In [4]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_case as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.CLIPS


In [5]:
models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    # train.Metrics.MEAN_SCL, 
    # train.Metrics.SCR_RATE, 
# ]
] + train.Metrics.STATISTICAL

In [6]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


random.seed(10)

model_phases = dr.CLIPS

label_type = dr.SelfReports.AROUSAL
threshold = "fixed"

x, y = train.Train_CASE.get_case_data(
    metrics, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False, standardize=True
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    # features = {name: metrics for name in models.keys()}
    features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.73407922 0.73207239 0.72846214 0.7341967  0.73478821 0.71498435
 0.73223911 0.7302545  0.73832342 0.73848547 0.72911322 0.74394233
 0.74430913 0.75072502 0.75226924 0.70754198 0.72927101 0.73883375
 0.75194501 0.74635336 0.72513589 0.73585539 0.74219152 0.75836901
 0.7481322         nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 1, 'kernel': 'poly'}
LGB: {'max_depth': 6, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 30, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [36 12], Predictions: [0 1], [46  2]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [36 12], Predictions: [0 1], [35 13]
Training RF ...
Model RF, Actual: [0 1], [36 12], Predictions: [0 1], [39  9]
Training XGB ...
Model XGB, Actual: [0 1], [36 12], Predictions: [0 1], [33 15]

None


[('lf_rr', 181), ('rmssd', 178), ('bpm', 169), ('hf_rr', 167), ('sdnn', 153)]




One or more of the test scores are non-finite: [0.77784865 0.78284001 0.78617602 0.7913156  0.79625352 0.75407183
 0.78174218 0.78794434 0.79934264 0.79860243 0.78184473 0.7812188
 0.7908117  0.79829562 0.79344253 0.78186085 0.7868918  0.79862822
 0.80032535 0.80076354 0.78380946 0.78535107 0.79325644 0.79870253
 0.79480964        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 0.01, 'kernel': 'poly'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [38 10], Predictions: [0], [48]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [38 10], Predictions: [0 1], [41  7]
Training RF ...
Model RF, Actual: [0 1], [38 10], Predictions: [0 1], [38 10]
Training XGB ...
Model XGB, Actual: [0 1], [38 10], Predictions: [0 1], [39  9]

None


[('bpm', 166), ('rmssd', 166), ('sdnn', 128), ('hf_rr', 88), ('lf_rr', 72)]


[('b

...

Grid search for XGB ...
SVM: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
LGB: {'max_depth': 6, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 6, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [34 14], Predictions: [0 1], [46  2]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [34 14], Predictions: [0 1], [41  7]
Training RF ...
Model RF, Actual: [0 1], [34 14], Predictions: [0 1], [44  4]
Training XGB ...
Model XGB, Actual: [0 1], [34 14], Predictions: [0 1], [44  4]

None


[('bpm', 260), ('hf_rr', 209), ('sdnn', 202), ('lf_rr', 163), ('rmssd', 139)]




One or more of the test scores are non-finite: [0.73117271 0.75688736 0.77366304 0.77668834 0.78302688 0.76975338
 0.76218623 0.76884143 0.7694344  0.77277559 0.74763901 0.77009848
 0.77668151 0.77272345 0.77578796 0.75673494 0.75961397 0.77040693
 0.76786571 0.76517666 0.75476169 0.76473676 0.77178833 0.77479461
 0.775763          nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [35 13], Predictions: [0 1], [45  3]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [35 13], Predictions: [0 1], [31 17]
Training RF ...
Model RF, Actual: [0 1], [35 13], Predictions: [0 1], [37 11]
Training XGB ...
Model XGB, Actual: [0 1], [35 13], Predictions: [0 1], [38 10]

None


[('rmssd', 229), ('bpm', 198), ('hf_rr', 173), ('lf_rr', 137), ('sdnn', 137)

One or more of the test scores are non-finite: [0.72770029 0.73696027 0.74978012 0.75594051 0.75760498 0.7089588
 0.72507935 0.72887386 0.7388     0.73906097 0.73360958 0.74496257
 0.75011531 0.75296213 0.75452107 0.74602654 0.73321051 0.73955771
 0.74192684 0.74856997 0.75998916 0.75409157 0.7555284  0.760212
 0.75995451        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 0.01, 'kernel': 'poly'}
LGB: {'max_depth': 5, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 6, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Feature selection for SVM ...
Feature selection for LGB ...
Feature selection for RF ...
Feature selection for XGB ...
Training SVM ...
Model SVM, Actual: [0 1], [32 16], Predictions: [0 1], [45  3]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [32 16], Predictions: [0 1], [32 16]
Training RF ...
Model RF, Actual: [0 1], [32 16], Predictions: [0 1], [33 15]
Training XGB ...
Model XGB, Actual: [0 1], [32 16], Predictions: [0 1], [38 10]

None


[('rmssd', 179), ('lf_rr', 173), ('bpm', 166), ('hf_rr', 148), ('sdnn', 128)]

In [7]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

random.seed(42)

test_size = 0.15
folds = 5
ensemble_models = [
    # "SVM", 
    "LGB",
    "RF",
    "XGB"
]

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds)
y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = [(name, best_models[name]) for name in ensemble_models]
ensemble = VotingClassifier(estimators, voting="hard")
ensemble.fit(x_train, y_train)
y_pred = ensemble.predict(x_test)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

	Accuracy: 0.8
	Precision: 0.6875
	Recall: 0.5
	F1-score: 0.5789473684210527
	AUC score: 0.7068965517241379
----------------------------------------


In [8]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier

random.seed(42)

def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    features = list(features.values())[0]
    # y_preds = []
    # # print(features)
    # for model_name in ensemble_models:
    #     x_test_temp = x_test.loc[:, features[model_name]]
    #     y_pred = ensemble_models[model_name].predict(x_test_temp)
    #     y_preds.append(y_pred)
    # if type == "majority_vote":
    #     y_preds = mode(y_preds, axis=0)[0]
    #     y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    # elif type == "weighted_avg":
    #     if weights is None:
    #         weights = [1 for _ in y_preds]
    #     for i in range(len(y_preds)):
    #         y_preds[i] = y_preds[i]*weights[i]
    #     y_preds = np.rint(np.mean(y_preds, axis=0))

    ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
    ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
    ensemble.fit(x_train.loc[:, features], y_train)
    y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds


test_size = 0.15
folds = 5
ensemble_models = [
    # "SVM", 
    "LGB",
    "RF",
    "XGB"
]
voting_type = "majority_vote"
weights = [
    # 0.8,
    0.5,
    1.0,
    1.0
]

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds)
y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models[name] for name in ensemble_models}
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

	Accuracy: 0.7
	Precision: 0.5454545454545454
	Recall: 0.24
	F1-score: 0.3333333333333333
	AUC score: 0.5745454545454545
----------------------------------------
